### C-CrSe2

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import AutoMinorLocator

rcParams['font.family'] = 'Arial'

folder_path = r'planar-averaged_charge_density_difference\c-crse2'
x_file = os.path.join(folder_path, 'x_values.dat')
a = 13.5726508  
# a = 13.3568535
# a = 13.5312011
# a = 13.234567
# a = 13.5414134


x = np.loadtxt(x_file)
results = []

img_dir = os.path.join(folder_path, 'plots_after_Z_Δρ0')
os.makedirs(img_dir, exist_ok=True)



def split_regions_from_x_equals_a(x, y, a):
    start_idx = np.searchsorted(x, a)
    if x[start_idx] == a:
        y_a = y[start_idx]
    else:
        x0, x1 = x[start_idx - 1], x[start_idx]
        y0, y1 = y[start_idx - 1], y[start_idx]
        y_a = y0 + (y1 - y0) * (a - x0) / (x1 - x0)
    x_new = np.concatenate([[a], x[start_idx:]])
    y_new = np.concatenate([[y_a], y[start_idx:]])
    crossings = []
    for i in range(1, len(y_new)):
        if y_new[i - 1] * y_new[i] < 0:
            xc = x_new[i - 1] - y_new[i - 1] * (x_new[i] - x_new[i - 1]) / (y_new[i] - y_new[i - 1])
            crossings.append((i, xc))
    segments = []
    idx_list = [0] + [i for i, _ in crossings] + [len(x_new) - 1]
    x_cross = [a] + [xc for _, xc in crossings] + [x_new[-1]]

    for i in range(len(idx_list) - 1):
        i_start, i_end = idx_list[i], idx_list[i + 1]
        x_seg = list(x_new[i_start:i_end])
        y_seg = list(y_new[i_start:i_end])
        x_seg = [x_cross[i]] + x_seg
        y_seg = [0.0] + y_seg

        if i < len(idx_list) - 2:
            x_seg.append(x_cross[i + 1])
            y_seg.append(0.0)

        segments.append((np.array(x_seg), np.array(y_seg)))
    return segments
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.dat') and filename != 'x_values.dat':
        file_path = os.path.join(folder_path, filename)
        y = np.loadtxt(file_path, skiprows=1)

        segments = split_regions_from_x_equals_a(x, y, a)

        pos_areas = []
        neg_areas = []
        for x_seg, y_seg in segments:
            area = simpson(y_seg, x_seg)
            if area > 0:
                pos_areas.append((area, x_seg, y_seg))
            elif area < 0:
                neg_areas.append((area, x_seg, y_seg))
        pos_areas.sort(key=lambda x: -x[0])
        neg_areas.sort(key=lambda x: x[0])
        result = {'File': filename}
        for i, (area, _, _) in enumerate(pos_areas):
            result[f'Charge Accumulation {i + 1}'] = area
        for i, (area, _, _) in enumerate(neg_areas):
            result[f'Charge Depletion {i + 1}'] = area
        results.append(result)
        
        custom_titles = {
    'charge_density_CrSe2-0.03.dat': r'C-CrSe$_2$, $E = -0.03$ V/Å',
    'charge_density_CrSe2-0.06.dat': r'C-CrSe$_2$, $E = -0.06$ V/Å',
    'charge_density_CrSe2-0.09.dat': r'C-CrSe$_2$, $E = -0.09$ V/Å',
    'charge_density_CrSe2-0.1.dat': r'C-CrSe$_2$, $E = -0.1$ V/Å',
    'charge_density_CrSe2-0.12.dat': r'C-CrSe$_2$, $E = -0.12$ V/Å',
    'charge_density_CrSe2-0.15.dat': r'C-CrSe$_2$, $E = -0.15$ V/Å',
    'charge_density_CrSe2-0.18.dat': r'C-CrSe$_2$, $E = -0.18$ V/Å',
    'charge_density_CrSe2-0.2.dat': r'C-CrSe$_2$, $E = -0.2$ V/Å',
    'charge_density_CrSe2-0.21.dat': r'C-CrSe$_2$, $E = -0.21$ V/Å',
    'charge_density_CrSe2-0.24.dat': r'C-CrSe$_2$, $E = -0.24$ V/Å',
    'charge_density_CrSe2-0.27.dat': r'C-CrSe$_2$, $E = -0.27$ V/Å',
    'charge_density_CrSe2-0.3.dat': r'C-CrSe$_2$, $E = -0.3$ V/Å',
    'charge_density_CrSe2-0.4.dat': r'C-CrSe$_2$, $E = -0.4$ V/Å',
    'charge_density_CrSe2-0.5.dat': r'C-CrSe$_2$, $E = -0.5$ V/Å',
    'charge_density_CrSe20.0.dat': r'C-CrSe$_2$, $E = -0.0$ V/Å',
    'charge_density_CrSe20.1.dat': r'C-CrSe$_2$, $E = 0.1$ V/Å',
    'charge_density_CrSe20.2.dat': r'C-CrSe$_2$, $E = 0.2$ V/Å',
    'charge_density_CrSe20.3.dat': r'C-CrSe$_2$, $E = 0.3$ V/Å',
    'charge_density_CrSe20.4.dat': r'C-CrSe$_2$, $E = 0.4$ V/Å',
    'charge_density_CrSe20.5.dat': r'C-CrSe$_2$, $E = 0.5$ V/Å',
}


      
        plt.figure(figsize=(14.86, 8), dpi=600)
        plt.plot(x, y, color='black')
        first_pos = True
        first_neg = True
        for (_, x_fill, y_fill) in neg_areas:
            plt.fill_between(x_fill, y_fill, color='blue', alpha=0.3,
                             label='Charge Depletion' if first_neg else "")
            first_neg = False
        for (_, x_fill, y_fill) in pos_areas:
            plt.fill_between(x_fill, y_fill, color='red', alpha=0.3,
                             label='Charge Accumulation' if first_pos else "")
            first_pos = False
        plt.axhline(0, color='gray', linestyle='--', linewidth=2)
        plt.axvline(a, color='black', linestyle='-.', label = f'a = {a:.3f} Å', linewidth=2)
        plt.xlabel('Z (Å)', fontsize=36)
        plt.ylabel('Δρ (e/Å)', fontsize=36)
        plt.tick_params(axis='both', labelsize=30)
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(3) 
        
        plt.ylim(-0.3, 0.3)
        plt.xlim(0, 26.5848) 
        plt.yticks(np.arange(-0.3, 0.4, 0.1))
        plt.xticks(np.arange(0, 26.5848, 3))
        plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))
        plt.tick_params(axis='both',    
                which='major',  
                length=7,        
                width=3,         
                direction='out')
        plt.tick_params(axis='both',     
                which='minor',
                length=4,        
                width=3,         
                direction='out')
        plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))  
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))
        plt.title(f"{custom_titles.get(filename, filename)}, Integration from Z = {a:.3f} Å", fontsize=30, pad=15)
        plt.legend(ncol=1, fontsize=24)
        plt.tight_layout()
        plt.savefig(os.path.join(img_dir, filename.replace('.dat', '.png')))
        plt.close()


df = pd.DataFrame(results)
excel_path = os.path.join(folder_path, 'results_of_Integral_from_Z=a_and_Δρ=0_all_peaks.xlsx')
df.to_excel(excel_path, index=False)


### C-MoS2

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from matplotlib import rcParams

# 设置全局字体
rcParams['font.family'] = 'Arial'

# 参数设置
folder_path = r'planar-averaged_charge_density_difference\c-mos2'
x_file = os.path.join(folder_path, 'x_values.dat')
# a = 13.5726508
a = 13.3568535
# a = 13.5312011
# a = 13.234567
# a = 13.5414134
x = np.loadtxt(x_file)
results = []


img_dir = os.path.join(folder_path, 'plots_after_Z_Δρ0')
os.makedirs(img_dir, exist_ok=True)


def split_regions_from_x_equals_a(x, y, a):
  
    start_idx = np.searchsorted(x, a)

   
    if x[start_idx] == a:
        y_a = y[start_idx]
    else:
        x0, x1 = x[start_idx - 1], x[start_idx]
        y0, y1 = y[start_idx - 1], y[start_idx]
        y_a = y0 + (y1 - y0) * (a - x0) / (x1 - x0)

   
    x_new = np.concatenate([[a], x[start_idx:]])
    y_new = np.concatenate([[y_a], y[start_idx:]])

    crossings = []
    for i in range(1, len(y_new)):
        if y_new[i - 1] * y_new[i] < 0:
            xc = x_new[i - 1] - y_new[i - 1] * (x_new[i] - x_new[i - 1]) / (y_new[i] - y_new[i - 1])
            crossings.append((i, xc))

  
    segments = []
    idx_list = [0] + [i for i, _ in crossings] + [len(x_new) - 1]
    x_cross = [a] + [xc for _, xc in crossings] + [x_new[-1]]

    for i in range(len(idx_list) - 1):
        i_start, i_end = idx_list[i], idx_list[i + 1]
        x_seg = list(x_new[i_start:i_end])
        y_seg = list(y_new[i_start:i_end])

       
        x_seg = [x_cross[i]] + x_seg
        y_seg = [0.0] + y_seg

      
        if i < len(idx_list) - 2:
            x_seg.append(x_cross[i + 1])
            y_seg.append(0.0)

        segments.append((np.array(x_seg), np.array(y_seg)))
    return segments



for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.dat') and filename != 'x_values.dat':
        file_path = os.path.join(folder_path, filename)
        y = np.loadtxt(file_path, skiprows=1)

        segments = split_regions_from_x_equals_a(x, y, a)

        pos_areas = []
        neg_areas = []
        for x_seg, y_seg in segments:
            area = simpson(y_seg, x_seg)
            if area > 0:
                pos_areas.append((area, x_seg, y_seg))
            elif area < 0:
                neg_areas.append((area, x_seg, y_seg))

       
        pos_areas.sort(key=lambda x: -x[0])
        neg_areas.sort(key=lambda x: x[0])

      
        result = {'File': filename}
        for i, (area, _, _) in enumerate(pos_areas):
            result[f'Charge Accumulation {i + 1}'] = area
        for i, (area, _, _) in enumerate(neg_areas):
            result[f'Charge Depletion {i + 1}'] = area
        results.append(result)
        
        custom_titles = {
            'charge_density_MoS2-0.04.dat':r'C-MoS$_2$, $E = -0.04$ V/Å', 
            'charge_density_MoS2-0.08.dat':r'C-MoS$_2$, $E = -0.08$ V/Å', 
            'charge_density_MoS2-0.1.dat':r'C-MoS$_2$, $E = -0.1$ V/Å', 
            'charge_density_MoS2-0.12.dat':r'C-MoS$_2$, $E = -0.12$ V/Å', 
            'charge_density_MoS2-0.16.dat':r'C-MoS$_2$, $E = -0.16$ V/Å', 
            'charge_density_MoS2-0.2.dat':r'C-MoS$_2$, $E = -0.2$ V/Å', 
            'charge_density_MoS2-0.24.dat':r'C-MoS$_2$, $E = -0.24$ V/Å', 
            'charge_density_MoS2-0.28.dat':r'C-MoS$_2$, $E = -0.28$ V/Å', 
            'charge_density_MoS2-0.3.dat':r'C-MoS$_2$, $E = -0.3$ V/Å', 
            'charge_density_MoS2-0.32.dat':r'C-MoS$_2$, $E = -0.32$ V/Å', 
            'charge_density_MoS2-0.36.dat':r'C-MoS$_2$, $E = -0.36$ V/Å', 
            'charge_density_MoS2-0.4.dat':r'C-MoS$_2$, $E = -0.4$ V/Å', 
            'charge_density_MoS2-0.5.dat':r'C-MoS$_2$, $E = -0.5$ V/Å', 
            'charge_density_MoS20.0.dat':r'C-MoS$_2$, $E = 0.0$ V/Å', 
            'charge_density_MoS20.1.dat':r'C-MoS$_2$, $E = 0.1$ V/Å', 
            'charge_density_MoS20.2.dat':r'C-MoS$_2$, $E = 0.2$ V/Å', 
            'charge_density_MoS20.3.dat':r'C-MoS$_2$, $E = 0.3$ V/Å', 
            'charge_density_MoS20.4.dat':r'C-MoS$_2$, $E = 0.4$ V/Å', 
            'charge_density_MoS20.5.dat':r'C-MoS$_2$, $E = 0.5$ V/Å'
    

}


        plt.figure(figsize=(14.86, 8), dpi=600)
        plt.plot(x, y, color='black')
        first_pos = True
        first_neg = True
        for (_, x_fill, y_fill) in neg_areas:
            plt.fill_between(x_fill, y_fill, color='blue', alpha=0.3,
                             label='Charge Depletion' if first_neg else "")
            first_neg = False
        for (_, x_fill, y_fill) in pos_areas:
            plt.fill_between(x_fill, y_fill, color='red', alpha=0.3,
                             label='Charge Accumulation' if first_pos else "")
            first_pos = False
        plt.axhline(0, color='gray', linestyle='--',  linewidth=2)
        plt.axvline(a, color='black', linestyle='-.', linewidth=2, label = f'a = {a:.3f} Å')
        plt.xlabel('Z (Å)', fontsize=36)
        plt.ylabel('Δρ (e/Å)', fontsize=36)
        plt.tick_params(axis='both', labelsize=30)
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(3) 
       
        plt.ylim(-0.25, 0.2)
        plt.xlim(0, 26.4525) 
        plt.yticks(np.arange(-0.25, 0.21, 0.05))
        plt.xticks(np.arange(0, 26.4525, 3))
        plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))
        plt.tick_params(axis='both',    
                which='major',   
                length=7,       
                width=3,         
                direction='out')
        plt.tick_params(axis='both',     
                which='minor',
                length=4,        
                width=3,         
                direction='out')
        plt.title(f"{custom_titles.get(filename, filename)}, Integration from Z = {a:.3f} Å", fontsize=30, pad=15)
        plt.legend(ncol=1, fontsize=24)
        plt.tight_layout()
        plt.savefig(os.path.join(img_dir, filename.replace('.dat', '.png')))
        plt.close()

df = pd.DataFrame(results)
excel_path = os.path.join(folder_path, 'results_of_Integral_from_Z=a_and_Δρ=0_all_peaks.xlsx')
df.to_excel(excel_path, index=False)


### C-MoSe2

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import AutoMinorLocator

rcParams['font.family'] = 'Arial'

folder_path = r'C:\Users\qianmen\Desktop\npj\planar-averaged_charge_density_difference\c-mose2'
x_file = os.path.join(folder_path, 'x_values.dat')
# a = 13.5726508 
# a = 13.3568535
a = 13.5312011
# a = 13.234567
# a = 13.5414134


x = np.loadtxt(x_file)
results = []

img_dir = os.path.join(folder_path, 'plots_after_base_Z_Δρ0')
os.makedirs(img_dir, exist_ok=True)

def split_regions_from_x_equals_a(x, y, a):
    
    start_idx = np.searchsorted(x, a)

    
    if x[start_idx] == a:
        y_a = y[start_idx]
    else:
        x0, x1 = x[start_idx - 1], x[start_idx]
        y0, y1 = y[start_idx - 1], y[start_idx]
        y_a = y0 + (y1 - y0) * (a - x0) / (x1 - x0)

    
    x_new = np.concatenate([[a], x[start_idx:]])
    y_new = np.concatenate([[y_a], y[start_idx:]])

    
    crossings = []
    for i in range(1, len(y_new)):
        if y_new[i - 1] * y_new[i] < 0:
            xc = x_new[i - 1] - y_new[i - 1] * (x_new[i] - x_new[i - 1]) / (y_new[i] - y_new[i - 1])
            crossings.append((i, xc))

    
    segments = []
    idx_list = [0] + [i for i, _ in crossings] + [len(x_new) - 1]
    x_cross = [a] + [xc for _, xc in crossings] + [x_new[-1]]

    for i in range(len(idx_list) - 1):
        i_start, i_end = idx_list[i], idx_list[i + 1]
        x_seg = list(x_new[i_start:i_end])
        y_seg = list(y_new[i_start:i_end])

       
        x_seg = [x_cross[i]] + x_seg
        y_seg = [0.0] + y_seg

       
        if i < len(idx_list) - 2:
            x_seg.append(x_cross[i + 1])
            y_seg.append(0.0)

        segments.append((np.array(x_seg), np.array(y_seg)))
    return segments



for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.dat') and filename != 'x_values.dat':
        file_path = os.path.join(folder_path, filename)
        y = np.loadtxt(file_path, skiprows=1)

        segments = split_regions_from_x_equals_a(x, y, a)

        pos_areas = []
        neg_areas = []
        for x_seg, y_seg in segments:
            area = simpson(y_seg, x_seg)
            if area > 0:
                pos_areas.append((area, x_seg, y_seg))
            elif area < 0:
                neg_areas.append((area, x_seg, y_seg))

       
        pos_areas.sort(key=lambda x: -x[0])
        neg_areas.sort(key=lambda x: x[0])

        
        result = {'File': filename}
        for i, (area, _, _) in enumerate(pos_areas):
            result[f'Charge Accumulation {i + 1}'] = area
        for i, (area, _, _) in enumerate(neg_areas):
            result[f'Charge Depletion {i + 1}'] = area
        results.append(result)
        
        custom_titles = {'charge_density_MoSe2-0.05.dat':r'C-MoSe$_2$, $E = -0.05$ V/Å', 
                         'charge_density_MoSe2-0.1.dat':r'C-MoSe$_2$, $E = -0.1$ V/Å', 
                         'charge_density_MoSe2-0.12.dat':r'C-MoSe$_2$, $E = -0.12$ V/Å', 
                         'charge_density_MoSe2-0.19.dat':r'C-MoSe$_2$, $E = -0.19$ V/Å', 
                         'charge_density_MoSe2-0.2.dat':r'C-MoSe$_2$, $E = -0.2$ V/Å', 
                         'charge_density_MoSe2-0.26.dat':r'C-MoSe$_2$, $E = -0.26$ V/Å', 
                         'charge_density_MoSe2-0.3.dat':r'C-MoSe$_2$, $E = -0.3$ V/Å', 
                         'charge_density_MoSe2-0.33.dat':r'C-MoSe$_2$, $E = -0.33$ V/Å', 
                         'charge_density_MoSe2-0.4.dat':r'C-MoSe$_2$, $E = -0.4$ V/Å', 
                         'charge_density_MoSe2-0.5.dat':r'C-MoSe$_2$, $E = -0.5$ V/Å', 
                         'charge_density_MoSe20.0.dat':r'C-MoSe$_2$, $E = 0.0$ V/Å', 
                         'charge_density_MoSe20.02.dat':r'C-MoSe$_2$, $E = 0.02$ V/Å', 
                         'charge_density_MoSe20.09.dat':r'C-MoSe$_2$, $E = 0.09$ V/Å', 
                         'charge_density_MoSe20.1.dat':r'C-MoSe$_2$, $E = 0.1$ V/Å', 
                         'charge_density_MoSe20.16.dat':r'C-MoSe$_2$, $E = 0.16$ V/Å', 
                         'charge_density_MoSe20.2.dat':r'C-MoSe$_2$, $E = 0.2$ V/Å', 
                         'charge_density_MoSe20.23.dat':r'C-MoSe$_2$, $E = 0.23$ V/Å', 
                         'charge_density_MoSe20.3.dat':r'C-MoSe$_2$, $E = 0.3$ V/Å', 
                         'charge_density_MoSe20.4.dat':r'C-MoSe$_2$, $E = 0.4$ V/Å', 
                         'charge_density_MoSe20.5.dat':r'C-MoSe$_2$, $E = 0.5$ V/Å'
}


     
        plt.figure(figsize=(14.86, 8), dpi=600)
        plt.plot(x, y, color='black')
        first_pos = True
        first_neg = True
        for (_, x_fill, y_fill) in neg_areas:
            plt.fill_between(x_fill, y_fill, color='blue', alpha=0.3,
                             label='Charge Depletion' if first_neg else "")
            first_neg = False
        for (_, x_fill, y_fill) in pos_areas:
            plt.fill_between(x_fill, y_fill, color='red', alpha=0.3,
                             label='Charge Accumulation' if first_pos else "")
            first_pos = False
        plt.axhline(0, color='gray', linestyle='--',  linewidth=2)
        plt.axvline(a, color='black', linestyle='-.', linewidth=2, label = f'a = {a:.3f} Å')
        plt.xlabel('Z (Å)', fontsize=36)
        plt.ylabel('Δρ (e/Å)', fontsize=36)
        plt.tick_params(axis='both', labelsize=30)
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(3)  # 比默认粗
           # 设置横轴范围为 10 到 20
        plt.ylim(-0.25, 0.2)
        plt.xlim(0, 26.8973) 
        plt.yticks(np.arange(-0.25, 0.21, 0.05))
        plt.xticks(np.arange(0, 26.8973, 3))
        plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))
        plt.tick_params(axis='both',    
                which='major',   
                length=7,        
                width=3,        
                direction='out')
        plt.tick_params(axis='both',     
                which='minor',
                length=4,        
                width=3,         
                direction='out')
        plt.title(f"{custom_titles.get(filename, filename)}, Integration from Z = {a:.3f} Å", fontsize=30, pad=15)
        plt.legend(ncol=1, fontsize=24)
        plt.tight_layout()
        plt.savefig(os.path.join(img_dir, filename.replace('.dat', '.png')))
        plt.close()

# 输出 Excel 文件
df = pd.DataFrame(results)
excel_path = os.path.join(folder_path, 'results_of_Integral_from_Z=a_and_Δρ=0_all_peaks.xlsx')
df.to_excel(excel_path, index=False)
print(f"积分完成，所有区域已保存至：{excel_path}")


### C-WS2

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['font.family'] = 'Arial'


folder_path = r'C:\Users\qianmen\Desktop\npj\planar-averaged_charge_density_difference\c-ws2'
x_file = os.path.join(folder_path, 'x_values.dat')
# a = 13.5726508 
# a = 13.3568535
# a = 13.5312011
a = 13.234567
# a = 13.5414134

x = np.loadtxt(x_file)
results = []


img_dir = os.path.join(folder_path, 'plots_after_base_Z_Δρ0')
os.makedirs(img_dir, exist_ok=True)


def split_regions_from_x_equals_a(x, y, a):
    
    start_idx = np.searchsorted(x, a)

   
    if x[start_idx] == a:
        y_a = y[start_idx]
    else:
        x0, x1 = x[start_idx - 1], x[start_idx]
        y0, y1 = y[start_idx - 1], y[start_idx]
        y_a = y0 + (y1 - y0) * (a - x0) / (x1 - x0)

    x_new = np.concatenate([[a], x[start_idx:]])
    y_new = np.concatenate([[y_a], y[start_idx:]])

   
    crossings = []
    for i in range(1, len(y_new)):
        if y_new[i - 1] * y_new[i] < 0:
            xc = x_new[i - 1] - y_new[i - 1] * (x_new[i] - x_new[i - 1]) / (y_new[i] - y_new[i - 1])
            crossings.append((i, xc))

   
    segments = []
    idx_list = [0] + [i for i, _ in crossings] + [len(x_new) - 1]
    x_cross = [a] + [xc for _, xc in crossings] + [x_new[-1]]

    for i in range(len(idx_list) - 1):
        i_start, i_end = idx_list[i], idx_list[i + 1]
        x_seg = list(x_new[i_start:i_end])
        y_seg = list(y_new[i_start:i_end])

      
        x_seg = [x_cross[i]] + x_seg
        y_seg = [0.0] + y_seg

     
        if i < len(idx_list) - 2:
            x_seg.append(x_cross[i + 1])
            y_seg.append(0.0)

        segments.append((np.array(x_seg), np.array(y_seg)))
    return segments



for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.dat') and filename != 'x_values.dat':
        file_path = os.path.join(folder_path, filename)
        y = np.loadtxt(file_path, skiprows=1)

        segments = split_regions_from_x_equals_a(x, y, a)

        pos_areas = []
        neg_areas = []
        for x_seg, y_seg in segments:
            area = simpson(y_seg, x_seg)
            if area > 0:
                pos_areas.append((area, x_seg, y_seg))
            elif area < 0:
                neg_areas.append((area, x_seg, y_seg))

        pos_areas.sort(key=lambda x: -x[0])
        neg_areas.sort(key=lambda x: x[0])

        result = {'File': filename}
        for i, (area, _, _) in enumerate(pos_areas):
            result[f'Charge Accumulation {i + 1}'] = area
        for i, (area, _, _) in enumerate(neg_areas):
            result[f'Charge Depletion {i + 1}'] = area
        results.append(result)
        custom_titles = {'charge_density_WS2-0.04.dat':r'C-WS$_2$, $E = -0.04$ V/Å', 
                         'charge_density_WS2-0.085.dat':r'C-WS$_2$, $E = -0.085$ V/Å', 
                         'charge_density_WS2-0.1.dat':r'C-WS$_2$, $E = -0.1$ V/Å', 
                         'charge_density_WS2-0.13.dat':r'C-WS$_2$, $E = -0.13$ V/Å', 
                         'charge_density_WS2-0.175.dat':r'C-WS$_2$, $E = -0.175$ V/Å', 
                         'charge_density_WS2-0.2.dat':r'C-WS$_2$, $E = -0.2$ V/Å', 
                         'charge_density_WS2-0.22.dat':r'C-WS$_2$, $E = -0.22$ V/Å', 
                         'charge_density_WS2-0.265.dat':r'C-WS$_2$, $E = -0.265$ V/Å', 
                         'charge_density_WS2-0.3.dat':r'C-WS$_2$, $E = -0.3$ V/Å', 
                         'charge_density_WS2-0.31.dat':r'C-WS$_2$, $E = -0.31$ V/Å', 
                         'charge_density_WS2-0.355.dat':r'C-WS$_2$, $E = -0.355$ V/Å', 
                         'charge_density_WS2-0.4.dat':r'C-WS$_2$, $E = -0.4$ V/Å', 
                         'charge_density_WS2-0.5.dat':r'C-WS$_2$, $E = -0.5$ V/Å', 
                         'charge_density_WS20.0.dat':r'C-WS$_2$, $E = 0.0$ V/Å', 
                         'charge_density_WS20.005.dat':r'C-WS$_2$, $E = 0.005$ V/Å', 
                         'charge_density_WS20.05.dat':r'C-WS$_2$, $E = 0.05$ V/Å', 
                         'charge_density_WS20.1.dat':r'C-WS$_2$, $E = 0.1$ V/Å', 
                         'charge_density_WS20.2.dat':r'C-WS$_2$, $E = 0.2$ V/Å', 
                         'charge_density_WS20.3.dat':r'C-WS$_2$, $E = 0.3$ V/Å', 
                         'charge_density_WS20.4.dat':r'C-WS$_2$, $E = 0.4$ V/Å', 
                         'charge_density_WS20.5.dat':r'C-WS$_2$, $E = 0.5$ V/Å'                     
}


      
        plt.figure(figsize=(14.86, 8), dpi=600)
        plt.plot(x, y, color='black')
        first_pos = True
        first_neg = True
        for (_, x_fill, y_fill) in neg_areas:
            plt.fill_between(x_fill, y_fill, color='blue', alpha=0.3,
                             label='Charge Depletion' if first_neg else "")
            first_neg = False
        for (_, x_fill, y_fill) in pos_areas:
            plt.fill_between(x_fill, y_fill, color='red', alpha=0.3,
                             label='Charge Accumulation' if first_pos else "")
            first_pos = False
        plt.axhline(0, color='gray', linestyle='--',  linewidth=2)
        plt.axvline(a, color='black', linestyle='-.', linewidth=2, label = f'a = {a:.3f} Å')
        plt.xlabel('Z (Å)', fontsize=36)
        plt.ylabel('Δρ (e/Å)', fontsize=36)
        plt.tick_params(axis='both', labelsize=30)
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(3) 
        plt.ylim(-0.25, 0.2)
        plt.xlim(0, 26.4762) 
        plt.yticks(np.arange(-0.25, 0.21, 0.05))
        plt.xticks(np.arange(0, 26.4762, 3))
        plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))
        plt.tick_params(axis='both',     
                which='major',   
                length=7,        
                width=3,        
                direction='out')
        plt.tick_params(axis='both',     
                which='minor',
                length=4,        
                width=3,         
                direction='out')
        plt.title(f"{custom_titles.get(filename, filename)}, Integration from Z = {a:.3f} Å", fontsize=30, pad=15)
        plt.legend(ncol=1, fontsize=24)
        plt.tight_layout()
        plt.savefig(os.path.join(img_dir, filename.replace('.dat', '.png')))
        plt.close()

df = pd.DataFrame(results)
excel_path = os.path.join(folder_path, 'results_of_Integral_from_Z=a_and_Δρ=0_all_peaks.xlsx')
df.to_excel(excel_path, index=False)


### C-WSe2

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from matplotlib import rcParams


rcParams['font.family'] = 'Arial'

folder_path = r'C:\Users\qianmen\Desktop\npj\planar-averaged_charge_density_difference\c-wse2'
x_file = os.path.join(folder_path, 'x_values.dat')
# a = 13.5726508 
# a = 13.3568535
# a = 13.5312011
# a = 13.234567
a = 13.5414134


x = np.loadtxt(x_file)
results = []


img_dir = os.path.join(folder_path, 'plots_after_base_Z_Δρ0')
os.makedirs(img_dir, exist_ok=True)


def split_regions_from_x_equals_a(x, y, a):
  
    start_idx = np.searchsorted(x, a)

    if x[start_idx] == a:
        y_a = y[start_idx]
    else:
        x0, x1 = x[start_idx - 1], x[start_idx]
        y0, y1 = y[start_idx - 1], y[start_idx]
        y_a = y0 + (y1 - y0) * (a - x0) / (x1 - x0)

    x_new = np.concatenate([[a], x[start_idx:]])
    y_new = np.concatenate([[y_a], y[start_idx:]])

 
    crossings = []
    for i in range(1, len(y_new)):
        if y_new[i - 1] * y_new[i] < 0:
            xc = x_new[i - 1] - y_new[i - 1] * (x_new[i] - x_new[i - 1]) / (y_new[i] - y_new[i - 1])
            crossings.append((i, xc))

  
    segments = []
    idx_list = [0] + [i for i, _ in crossings] + [len(x_new) - 1]
    x_cross = [a] + [xc for _, xc in crossings] + [x_new[-1]]

    for i in range(len(idx_list) - 1):
        i_start, i_end = idx_list[i], idx_list[i + 1]
        x_seg = list(x_new[i_start:i_end])
        y_seg = list(y_new[i_start:i_end])

        x_seg = [x_cross[i]] + x_seg
        y_seg = [0.0] + y_seg

        if i < len(idx_list) - 2:
            x_seg.append(x_cross[i + 1])
            y_seg.append(0.0)

        segments.append((np.array(x_seg), np.array(y_seg)))
    return segments


for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.dat') and filename != 'x_values.dat':
        file_path = os.path.join(folder_path, filename)
        y = np.loadtxt(file_path, skiprows=1)

        segments = split_regions_from_x_equals_a(x, y, a)

        pos_areas = []
        neg_areas = []
        for x_seg, y_seg in segments:
            area = simpson(y_seg, x_seg)
            if area > 0:
                pos_areas.append((area, x_seg, y_seg))
            elif area < 0:
                neg_areas.append((area, x_seg, y_seg))

      
        pos_areas.sort(key=lambda x: -x[0])
        neg_areas.sort(key=lambda x: x[0])

        result = {'File': filename}
        for i, (area, _, _) in enumerate(pos_areas):
            result[f'Charge Accumulation {i + 1}'] = area
        for i, (area, _, _) in enumerate(neg_areas):
            result[f'Charge Depletion {i + 1}'] = area
        results.append(result)

        custom_titles = {'charge_density_WSe2-0.02.dat':r'C-WSe$_2$, $E = -0.02$ V/Å', 
                         'charge_density_WSe2-0.09.dat':r'C-WSe$_2$, $E = -0.09$ V/Å', 
                         'charge_density_WSe2-0.1.dat':r'C-WSe$_2$, $E = -0.1$ V/Å', 
                         'charge_density_WSe2-0.16.dat':r'C-WSe$_2$, $E = -0.16$ V/Å', 
                         'charge_density_WSe2-0.2.dat':r'C-WSe$_2$, $E = -0.2$ V/Å', 
                         'charge_density_WSe2-0.23.dat':r'C-WSe$_2$, $E = -0.23$ V/Å', 
                         'charge_density_WSe2-0.3.dat':r'C-WSe$_2$, $E = -0.3$ V/Å', 
                         'charge_density_WSe2-0.4.dat':r'C-WSe$_2$, $E = -0.4$ V/Å', 
                         'charge_density_WSe2-0.5.dat':r'C-WSe$_2$, $E = -0.5$ V/Å', 
                         'charge_density_WSe20.0.dat':r'C-WSe$_2$, $E = 0.0$ V/Å', 
                         'charge_density_WSe20.05.dat':r'C-WSe$_2$, $E = 0.05$ V/Å', 
                         'charge_density_WSe20.1.dat':r'C-WSe$_2$, $E = 0.1$ V/Å', 
                         'charge_density_WSe20.12.dat':r'C-WSe$_2$, $E = 0.12$ V/Å', 
                         'charge_density_WSe20.19.dat':r'C-WSe$_2$, $E = 0.19$ V/Å', 
                         'charge_density_WSe20.2.dat':r'C-WSe$_2$, $E = 0.2$ V/Å', 
                         'charge_density_WSe20.26.dat':r'C-WSe$_2$, $E = 0.26$ V/Å', 
                         'charge_density_WSe20.3.dat':r'C-WSe$_2$, $E = 0.3$ V/Å', 
                         'charge_density_WSe20.33.dat':r'C-WSe$_2$, $E = 0.33$ V/Å', 
                         'charge_density_WSe20.4.dat':r'C-WSe$_2$, $E = 0.4$ V/Å', 
                         'charge_density_WSe20.5.dat':r'C-WSe$_2$, $E = 0.5$ V/Å'
}


        plt.figure(figsize=(14.86, 8), dpi=600)
        plt.plot(x, y, color='black')
        first_pos = True
        first_neg = True
        for (_, x_fill, y_fill) in neg_areas:
            plt.fill_between(x_fill, y_fill, color='blue', alpha=0.3,
                             label='Charge Depletion' if first_neg else "")
            first_neg = False
        for (_, x_fill, y_fill) in pos_areas:
            plt.fill_between(x_fill, y_fill, color='red', alpha=0.3,
                             label='Charge Accumulation' if first_pos else "")
            first_pos = False
        plt.axhline(0, color='gray', linestyle='--',  linewidth=2)
        plt.axvline(a, color='black', linestyle='-.', linewidth=2, label = f'a = {a:.3f} Å')
        plt.xlabel('Z (Å)', fontsize=36)
        plt.ylabel('Δρ (e/Å)', fontsize=36)
        plt.tick_params(axis='both', labelsize=30)
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(3) 
          
        plt.ylim(-0.20, 0.2)
        plt.xlim(0, 26.8973) 
        plt.yticks(np.arange(-0.20, 0.21, 0.04))
        plt.xticks(np.arange(0, 26.8973, 3))
        plt.gca().xaxis.set_minor_locator(AutoMinorLocator(2))
        plt.gca().yaxis.set_minor_locator(AutoMinorLocator(2))
        plt.tick_params(axis='both',    
                which='major',   
                length=7,       
                width=3,        
                direction='out')
        plt.tick_params(axis='both',     
                which='minor',
                length=4,        
                width=3,         
                direction='out')
        plt.title(f"{custom_titles.get(filename, filename)}, Integration from Z = {a:.3f} Å", fontsize=30, pad=15)
        plt.legend(ncol=1, fontsize=24)
        plt.tight_layout()
        plt.savefig(os.path.join(img_dir, filename.replace('.dat', '.png')))
        plt.close()


df = pd.DataFrame(results)
excel_path = os.path.join(folder_path, 'results_of_Integral_from_Z=a_and_Δρ=0_all_peaks.xlsx')
df.to_excel(excel_path, index=False)
